In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv('aadhaar_demo_monthly_update_cleaned.csv')

In [3]:
df.head()

,Date,State,District,Pincode,Demo_age_5_17,Demo_age_17+
0,01-03-2025,West Bengal,Murshidabad,742402,15,165
1,01-03-2025,Manipur,Thoubal,795101,12,114
2,01-03-2025,Rajasthan,Sirohi,307513,18,201
3,01-03-2025,West Bengal,Hooghly,712311,27,480
4,01-03-2025,Himachal Pradesh,Una,174301,24,243


#### Adding Features and Refining

In [4]:
cols = ["Demo_age_5_17", "Demo_age_17+"]
df[cols].dtypes

Demo_age_5_17    object
Demo_age_17+     object
dtype: object

In [5]:
df[cols] = df[cols].apply(pd.to_numeric, errors="coerce") 

total_pop = df[cols].sum().sum()

print(float(total_pop))

46321955.0


In [6]:
df["Date"].unique()

array(['01-03-2025', '01-04-2025', '01-05-2025', '01-06-2025',
       '01-07-2025', '01-09-2025', '02-09-2025', '03-09-2025',
       '04-09-2025', '05-09-2025', '06-09-2025', '07-09-2025',
       '08-09-2025', '09-09-2025', '10-09-2025', '12-09-2025',
       '11-09-2025', '13-09-2025', '15-09-2025', '14-09-2025',
       '16-09-2025', '17-09-2025', '18-09-2025', '19-09-2025',
       '20-09-2025', '13-10-2025', '15-10-2025', 'Date', '16-10-2025',
       '17-10-2025', '26-10-2025', '18-10-2025', '19-10-2025',
       '20-10-2025', '21-10-2025', '22-10-2025', '23-10-2025',
       '24-10-2025', '25-10-2025', '27-10-2025', '28-10-2025',
       '29-10-2025', '30-10-2025', '31-10-2025', '01-11-2025',
       '02-11-2025', '03-11-2025', '04-11-2025', '05-11-2025',
       '06-11-2025', '07-11-2025', '08-11-2025', '09-11-2025',
       '10-11-2025', '11-11-2025', '12-11-2025', '13-11-2025',
       '14-11-2025', '15-11-2025', '16-11-2025', '17-11-2025',
       '19-11-2025', '18-11-2025', '01-12-2025'

In [7]:
df["Date"] = pd.to_datetime(df["Date"], errors="coerce",dayfirst=True)  # parse to datetime
df["Month"] = df["Date"].dt.to_period("M")

In [8]:
df.head()

,Date,State,District,Pincode,Demo_age_5_17,Demo_age_17+,Month
0,2025-03-01,West Bengal,Murshidabad,742402,15.0,165.0,2025-03
1,2025-03-01,Manipur,Thoubal,795101,12.0,114.0,2025-03
2,2025-03-01,Rajasthan,Sirohi,307513,18.0,201.0,2025-03
3,2025-03-01,West Bengal,Hooghly,712311,27.0,480.0,2025-03
4,2025-03-01,Himachal Pradesh,Una,174301,24.0,243.0,2025-03


In [11]:
freq = df["State"].value_counts(dropna=False).reset_index()
freq.head(40)

,State,count
0,Tamil Nadu,232944
1,Andhra Pradesh,224865
2,Uttar Pradesh,196888
3,West Bengal,189387
4,Maharashtra,179383
5,Karnataka,177489
6,Odisha,121673
7,Kerala,118574
8,Gujarat,107116
9,Telangana,106119


In [10]:
threshold = 30

vc = df["State"].value_counts(dropna=False)
rare_vals = vc[vc.lt(threshold)].index

df.drop(df[df["State"].isin(rare_vals)].index,inplace=True)

In [12]:
old_states1 = ["Daman & Diu", "Dadra & Nagar Haveli"]
new_state1  = "Dadra and Nagar Haveli and Daman and Diu"

key = ["Date", "Pincode"]                 # your “same reading” definition
sum_cols = ["Demo_age_5_17", "Demo_age_17+"]
keep_cols = ["Month"]                     # keep from the canonical row
# District should store “where it originally came from”
# (for rows that were old_states, District becomes that old state name)
# (for already-new_state rows, keep existing District)

# 0) Ensure numeric
df[sum_cols] = df[sum_cols].apply(pd.to_numeric, errors="coerce").fillna(0)

# 1) Split
df_old  = df[df["State"].isin(old_states1)].copy()
df_new  = df[df["State"].eq(new_state1)].copy()
df_rest = df[~(df["State"].isin(old_states1) | df["State"].eq(new_state1))].copy()

# 2) Old rows: store original state in District, rename State -> new_state
df_old["District"] = df_old["State"]
df_old["State"] = new_state1

# 3) Collapse duplicates *within each side* (prevents many-to-many inflation)
#    - old side: District is meaningful; keep it (if multiple, join them)
old_agg = (df_old
    .groupby(key, as_index=False)
    .agg({
        "Demo_age_5_17": "sum",
        "Demo_age_17+": "sum",
        "Month": "first",
        "State": "first",
        "District": lambda s: ",".join(sorted(set(s.dropna().astype(str))))
    })
)

#    - new side: keep Month/District as-is (pick first non-null District)
def first_non_null(s):
    s2 = s.dropna()
    return s2.iloc[0] if len(s2) else np.nan

new_agg = (df_new
    .groupby(key, as_index=False)
    .agg({
        "Demo_age_5_17": "sum",
        "Demo_age_17+": "sum",
        "Month": "first",
        "State": "first",
        "District": first_non_null
    })
)

# 4) Transfer-then-delete logic:
#    If a key exists in new_agg, add old_agg counts into it and drop that old key.
merged = new_agg.merge(
    old_agg[key + sum_cols + ["District"]], on=key, how="left", suffixes=("", "_old")
)

for c in sum_cols:
    merged[c] = merged[c] + merged[f"{c}_old"].fillna(0)

# If new District is NaN, fill it from old; otherwise keep new's District
merged["District"] = merged["District"].fillna(merged["District_old"])

merged = merged.drop(columns=[f"{c}_old" for c in sum_cols] + ["District_old"])

# 5) Keep old-only keys that didn't exist in new_agg (no match → keep as-is)
old_only = old_agg.merge(new_agg[key], on=key, how="left", indicator=True)
old_only = old_only[old_only["_merge"].eq("left_only")].drop(columns=["_merge"])

# 6) Final UT block + stitch back with rest
ut_final = pd.concat([merged, old_only], ignore_index=True)

df_final = pd.concat([df_rest, ut_final], ignore_index=True)

In [13]:
new_name = "Dadra and Nagar Haveli and Daman and Diu"

# 1) All distinct (Date, Pincode) combos + how many rows each has
combo_counts = (
    df.loc[df["State"].eq(new_name), ["Date", "Pincode"]]
      .groupby(["Date", "Pincode"])
      .size()                             # count rows in each combination
      .reset_index(name="n_rows")         # turn into a DataFrame
)

# 2) Number of distinct dates
n_dates = df.loc[df["State"].eq(new_name), "Date"].nunique()

# 3) Number of distinct pincodes
n_pincodes = df.loc[df["State"].eq(new_name), "Pincode"].nunique()

In [14]:
len(combo_counts)

784

In [15]:
freq1 = df_final["State"].value_counts(dropna=False).reset_index()
freq1.head(40)

,State,count
0,Tamil Nadu,232944
1,Andhra Pradesh,224865
2,Uttar Pradesh,196888
3,West Bengal,189387
4,Maharashtra,179383
5,Karnataka,177489
6,Odisha,121673
7,Kerala,118574
8,Gujarat,107116
9,Telangana,106119


In [16]:
print(df[["Demo_age_5_17","Demo_age_17+"]].sum())
print(df_final[["Demo_age_5_17","Demo_age_17+"]].sum())

Demo_age_5_17     4242797.0
Demo_age_17+     42079119.0
dtype: float64
Demo_age_5_17     4242797.0
Demo_age_17+     42079119.0
dtype: float64


In [17]:
# Old dataset: sum of age demos where State is one of the three names
old_states2 = ["Daman & Diu","Dadra & Nagar Haveli",new_state1]

old_sum = (
    df[df["State"].isin(old_states2)][["Demo_age_5_17","Demo_age_17+"]]
    .sum()
)

# Final dataset: sum of age demos where State is the new merged name
new_sum = (
    df_final[df_final["State"]==new_state1][["Demo_age_5_17","Demo_age_17+"]]
    .sum()
)

print("Original (three names):")
print(old_sum)
print("\nFinal (merged name):")
print(new_sum)

print("\nDifference:")
print(new_sum - old_sum)

Original (three names):
Demo_age_5_17     1763.0
Demo_age_17+     10641.0
dtype: float64

Final (merged name):
Demo_age_5_17     1763.0
Demo_age_17+     10641.0
dtype: float64

Difference:
Demo_age_5_17    0.0
Demo_age_17+     0.0
dtype: float64


In [18]:
df_final.shape

(2337124, 7)

In [19]:
df = df_final

In [21]:
df_final.to_csv("aadhaar_demo_monthly_update_superclean.csv")

### Age based Calc

#### The below shows how many registered for Aadhaar sorted date wise

In [22]:
out2 = (df_final.groupby(["State","Month"])[cols].sum().reset_index().sort_values("State"))
out2.to_csv("Cleaned State Wise Data - Total Enrollments in 2025.csv",index=False)

out4 = (df_final.groupby(["State"])[cols].sum().reset_index().sort_values("State"))
out4.to_csv("State Wise Data - Total Enrollments in 2025.csv",index=False)

print(len(out2))
out2.head(60)

306


,State,Month,Demo_age_5_17,Demo_age_17+
0,Andaman and Nicobar Islands,2025-03,114.0,895.0
1,Andaman and Nicobar Islands,2025-09,93.0,1010.0
2,Andaman and Nicobar Islands,2025-10,62.0,611.0
3,Andaman and Nicobar Islands,2025-11,114.0,1708.0
4,Andaman and Nicobar Islands,2025-12,156.0,2059.0
5,Andaman and Nicobar Islands,2026-01,13.0,101.0
15,Andhra Pradesh,2026-01,3714.0,25522.0
14,Andhra Pradesh,2025-12,54331.0,421990.0
13,Andhra Pradesh,2025-11,61949.0,320340.0
12,Andhra Pradesh,2025-10,37112.0,175479.0


In [23]:
out1 = (df_final.groupby("Month")[cols].sum().reset_index().sort_values("Month"))
out1.head(20)

,Month,Demo_age_5_17,Demo_age_17+
0,2025-03,712681.0,7375637.0
1,2025-04,146659.0,988313.0
2,2025-05,158021.0,1038197.0
3,2025-06,151558.0,1087943.0
4,2025-07,203231.0,1355635.0
5,2025-09,895469.0,7535914.0
6,2025-10,413917.0,4039648.0
7,2025-11,673803.0,8634211.0
8,2025-12,821350.0,9409549.0
9,2026-01,66108.0,614072.0


In [24]:
out3 = (df_final.groupby("District")[cols].sum().reset_index().sort_values("District"))
out3.head()

,District,Demo_age_5_17,Demo_age_17+
0,ANGUL,2.0,9.0
1,ANUGUL,6.0,170.0
2,Adilabad,7190.0,61939.0
3,Agar Malwa,1042.0,9587.0
4,Agra,22164.0,205823.0


In [25]:
out_check = (df.groupby(["State","Month"])["District"].nunique().reset_index().sort_values("State"))

### Random Checks

In [26]:
all_combos = (
    df[["State", "District", "Month"]]
    .drop_duplicates()
)

# Step 2: build the full cartesian product of State–District with all Months
states_districts = df[["State", "District"]].drop_duplicates()
months = df["Month"].drop_duplicates()

expected = (
    states_districts.assign(key=1)
    .merge(months.to_frame().assign(key=1), on="key")
    .drop("key", axis=1)
)

# Step 3: find which combinations are missing
missing = expected.merge(all_combos, 
                         on=["State","District","Month"], 
                         how="left", 
                         indicator=True)

missing = missing[missing["_merge"] == "left_only"].drop("_merge", axis=1)

# Step 4: report
if missing.empty:
    print("✅ Every district was checked in every month")
else:
    print("❌ Some districts are missing in certain months")
    print(missing)


❌ Some districts are missing in certain months
                                         State              District    Month
11                                     Manipur               Thoubal  2025-04
12                                     Manipur               Thoubal  2025-05
13                                     Manipur               Thoubal  2025-06
14                                     Manipur               Thoubal  2025-07
21                                   Rajasthan                Sirohi  2025-04
...                                        ...                   ...      ...
9963  Dadra and Nagar Haveli and Daman and Diu  Dadra & Nagar Haveli  2025-06
9964  Dadra and Nagar Haveli and Daman and Diu  Dadra & Nagar Haveli  2025-07
9965  Dadra and Nagar Haveli and Daman and Diu  Dadra & Nagar Haveli  2025-09
9967  Dadra and Nagar Haveli and Daman and Diu  Dadra & Nagar Haveli  2025-11
9969  Dadra and Nagar Haveli and Daman and Diu  Dadra & Nagar Haveli  2026-01

[3120 rows x 3 c

In [27]:
# All unique states
states = df["State"].unique()

# All unique months
months = df["Month"].dropna().unique()

# Expected combinations: every state × every month
expected = pd.MultiIndex.from_product([states, months], names=["State","Month"]).to_frame(index=False)

# Actual combinations present
actual = df[["State","Month"]].drop_duplicates()

# Find missing state-month pairs
missing = expected.merge(actual, on=["State","Month"], how="left", indicator=True)
missing = missing[missing["_merge"]=="left_only"].drop("_merge", axis=1)

if missing.empty:
    print("✅ Every state occurs in every month")
else:
    print("❌ Some states are missing in certain months")
    print(missing.sort_values(["State","Month"]))


❌ Some states are missing in certain months
                                        State    Month
331               Andaman and Nicobar Islands  2025-04
332               Andaman and Nicobar Islands  2025-05
333               Andaman and Nicobar Islands  2025-06
334               Andaman and Nicobar Islands  2025-07
171                         Arunachal Pradesh  2025-04
172                         Arunachal Pradesh  2025-05
173                         Arunachal Pradesh  2025-06
174                         Arunachal Pradesh  2025-07
83                                      Bihar  2025-06
352  Dadra and Nagar Haveli and Daman and Diu  2025-05
353  Dadra and Nagar Haveli and Daman and Diu  2025-06
354  Dadra and Nagar Haveli and Daman and Diu  2025-07
251                                       Goa  2025-04
252                                       Goa  2025-05
253                                       Goa  2025-06
254                                       Goa  2025-07
223                  